In [1]:
import torch
from torch import optim
from torch import nn
from imblearn.over_sampling import SMOTE
import pandas as pd
from pathlib import Path

# Peep the raw data

In [2]:
raw_df = pd.read_csv(Path('./train.csv'))
display(raw_df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Drop irrelevant columns and deal with NaNs

In [3]:
# I'll drop the 'Name' and 'PassengerId' columns for now, as I suspect they have no bearing on survival
cleaned_df = raw_df.drop(columns=['PassengerId', 'Name'])
display(cleaned_df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [4]:
# I see a few NaNs
def count_nans(df: pd.DataFrame) -> None:
    "Prints a count of the occurences of NaNs in each column"
    for col in df.columns:
        print(f"{col} NaNs: {df[col].isna().sum()}")

In [5]:
count_nans(cleaned_df)

Survived NaNs: 0
Pclass NaNs: 0
Sex NaNs: 0
Age NaNs: 177
SibSp NaNs: 0
Parch NaNs: 0
Ticket NaNs: 0
Fare NaNs: 0
Cabin NaNs: 687
Embarked NaNs: 2


In [15]:
cleaned_df['Age'].fillna(cleaned_df['Age'].mean(), inplace=True)
cleaned_df['Cabin'].fillna('UNK', inplace=True)
cleaned_df['Embarked'].fillna('UNK', inplace=True)
count_nans(cleaned_df)

Survived NaNs: 0
Pclass NaNs: 0
Sex NaNs: 0
Age NaNs: 0
SibSp NaNs: 0
Parch NaNs: 0
Ticket NaNs: 0
Fare NaNs: 0
Cabin NaNs: 0
Embarked NaNs: 0


In [7]:
# I'm reusing my custom training loop
def training_loop(epochs, model, loss_fcn, optimizer, train_dataloader, val_dataloader,
                  *, save_best=True, metrics=True, logging=True):
    """
    Custom training loop
    Parameters:
        epochs: int, number of epochs to train for
        model: nn.Module or subclass thereof, from which to obtain predictions
        loss_fcn: any pytorch loss function
        optimizer: any optimizer
        train_dataloader: PyTorch dataloader from which to pull data
        val_dataloader: "
        save_best: bool, WARNING only use on smaller models, cache and serialize best model at end of training
        metrics: bool, control calculation and printing of numbers to the screen
        logging: bool, control printing of tensor info to screen after each step
    Returns: 
        Trained model
    """
    
    highest_accuracy = 0
    cached_model = None
    for epoch in range(epochs):
        for features, labels in train_dataloader:
            labels.unsqueeze_(1)
            if logging: print_t_info([features, labels], ["Train Feats: ", "Train Labels: "])
            
            train_predictions = model(features)
            
            train_loss = loss_fcn(train_predictions, labels)
            
            if logging: print_t_info([train_predictions, train_loss], ["Train Preds:", "Train Loss:"])
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            
        total = 0
        correct = 0
        
        for features, labels in val_dataloader:
            labels.unsqueeze_(1)
            if logging: print_t_info([features, labels], ['Val Feats: ', 'Val Labels:'])
            
            val_predictions = model(features)
            
            val_loss = loss_fcn(val_predictions, labels)
            
            if logging: print_t_info([val_predictions, val_loss], ['Val Preds:', 'Val Loss:'])
            
            break
            
        if metrics:
            total += val_predictions.shape[0]
            correct = int(((val_predictions > 0.5) == labels.type(torch.BoolTensor)).sum())
            print(f"Epoch {epoch:03}\tTrain Loss: {train_loss:.4}\tVal Loss: "
                  f"{val_loss:.4}\tAccuracy: {correct/total:%}")
        if save_best:
            latest_accuracy = correct/total
            if latest_accuracy > highest_accuracy:
                highest_accuracy = latest_accuracy
                cached_model = model
                
    model_pth = Path(f'./models/{highest_accuracy * 100:.5}_model')
    torch.save(cached_model, model_pth)
    print(f"Model saved to {model_pth}")
        
    return model